In [1]:
from opt_lib import *
# Parameter definieren
dt = 0.001
L = 0.5
v_max = 32

test = data_class("TrafficDataset/sequence_data1-1_1.mat")
test.next_dataset() # Nächster Datensatz
test.next_dataset() # Nächster Datensatz
test.next_dataset() # Nächster Datensatz
test.LinInterplation(dt) # mit dt schrittweite, denn Datensatz interpolieren
print(test.test_interpolation(dt)) # durchschnittlicher Fehler der Interpolation abfragen
test.switch_iteration() # Die interpolierten werte als zu berücksichtigende Daten setzen

x_num = np.zeros_like(test.get_x())# Array definieren, in dem die numerisch bestimmten ort abgespeichert werden
J_array = np.zeros(test.get_t().shape[0] - 1) # Array definieren, in dem der Loss abgespeichert wird

for i in range(len(J_array)+1):
	x_soll, t_soll = test.getCurrentCoords() # Aktuelle ort, zeit punkte der autos ausgeben
	if i != 0: # Falls wir schon einen Schritt gegangen sind
		J_array[i-1] = J(x, dt, x_soll) # Loss berechnen
	else: # Falls der erste durchlauf
		x, t = x_soll, t_soll
	dist = d(x) # Distanzen zwischen den autos berechnen
	x, _ = single_step(x, v_max, L, dt, "lin") # x aktualisieren
	x_num[:,i] = x
	t += dt

test.getCurrentCoords() # Test, ob alle Daten abgegangen wurden

[6.33294764e-13 5.54756678e-13 2.16477404e-13]


(array([], dtype=float64), None)

In [2]:
import pygame
pygame.init()
W, H = 900, 300
offset = 50
screen = pygame.display.set_mode((W + offset, H + offset))
clock = pygame.time.Clock()
scale = 10     # skaliert Meter → Pixel
running = True
time_lst = test.get_t()


WIDTH, HEIGHT = W, H
# Farben
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# Koordinatensystem-Einstellungen
ORIGIN_X = 100 # WIDTH // 2
ORIGIN_Y = 200 # HEIGHT // 2
GRID_SIZE = 50 # Abstand zwischen Gitterlinien

def draw_coordinate_system():
    """Zeichnet das statische Koordinatensystem"""
    # Gitterlinien
    for i in range(0, WIDTH, GRID_SIZE):
        # Vertikale Linien
        screen_x = ORIGIN_X + i
        pygame.draw.line(screen, GRAY, (screen_x, 0), (screen_x, HEIGHT), 1)
        
    # for i in range(0, HEIGHT, GRID_SIZE):
    #     # Horizontale Linien
    #     screen_y = ORIGIN_Y + i
    #     pygame.draw.line(screen, GRAY, (0, screen_y), (WIDTH, screen_y), 1)
    
    # Achsen (dicker)
    pygame.draw.line(screen, WHITE, (ORIGIN_X, 0), (ORIGIN_X, HEIGHT), 2)  # Y-Achse
    pygame.draw.line(screen, WHITE, (0, ORIGIN_Y), (WIDTH, ORIGIN_Y), 2)  # X-Achse
    
    # Beschriftungen
    font = pygame.font.Font(None, 20)
    
    # X-Achse Beschriftung
    for i in range(0, WIDTH, GRID_SIZE):
        if i != 0:
            screen_x = ORIGIN_X + i
            text = font.render(f"{i/scale_x: .3}", True, WHITE)
            screen.blit(text, (screen_x - 10, ORIGIN_Y + 5))
    
    # # Y-Achse Beschriftung (Y-Achse nach unten ist negativ)
    # for i in range(0, HEIGHT, GRID_SIZE):
    #     if i != 0:
    #         screen_y = ORIGIN_Y + i
    #         text = font.render(str(-i), True, WHITE)
    #         screen.blit(text, (ORIGIN_X + 5, screen_y - 10))
    
    # Ursprung
    text = font.render("0", True, WHITE)
    screen.blit(text, (ORIGIN_X + 5, ORIGIN_Y + 5))
def start_pos(x):
	pos = np.zeros_like(x)
	for i in range(pos.shape[0]):
		pos[i] = x[i] - np.min(x)
	return pos

xmins_n = np.min(x_num, 1)
xmins_data = np.min(test.get_x(), 1)
start_n = start_pos(xmins_n)
start_data = start_pos(xmins_data)

scale_x = (W-ORIGIN_X)/np.max((test.get_x()) - np.min(test.get_x())) # 
for x_n, x_data, time in zip(x_num.T, test.get_x().T, time_lst-np.min(time_lst)):
	clock.tick(60) # 60 FPS
	for event in pygame.event.get():
		if event.type == pygame.QUIT:
			running = False             # Stop when closed
	if not running:
		break			# Stop when closed
	
	screen.fill((30, 30, 30))
	draw_coordinate_system()
	for i, xn, xd in zip(range(len(x_n)), x_n, x_data): 
		pygame.draw.circle(screen, RED, ((xn - xmins_n[i]) * scale_x + start_n[i]* scale_x + ORIGIN_X, 70), 5) #cars
		pygame.draw.circle(screen, BLUE, ((xd - xmins_data[i]) * scale_x + start_data[i]* scale_x + ORIGIN_X, 150), 5) #cars

	pygame.display.flip() # display and render 
pygame.quit()

/home/tim/miniconda3/envs/Opt1/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
# mittelpunktsregel fpr das integral

def single_step_P(x, v_max, L, dt, model):
    N = len(x) # Number of initial values == number of vehicles in traffic
    dx = np.zeros(N) # momente initialisieren
    # Leader
    # dx[-1] = v_max
    dist = d(x)/L
    # for i in range(N):
    #         dx[i] = f(dist[i], v_max, model)
    dx = P(dist, v_max, model)
    return x + dt * dx, dx

def P():
    return
    
def adjungierte(xn, vmax, P):
    return 

In [4]:
import numpy as np
np.arange(10, 0, -0.1)

array([10. ,  9.9,  9.8,  9.7,  9.6,  9.5,  9.4,  9.3,  9.2,  9.1,  9. ,
        8.9,  8.8,  8.7,  8.6,  8.5,  8.4,  8.3,  8.2,  8.1,  8. ,  7.9,
        7.8,  7.7,  7.6,  7.5,  7.4,  7.3,  7.2,  7.1,  7. ,  6.9,  6.8,
        6.7,  6.6,  6.5,  6.4,  6.3,  6.2,  6.1,  6. ,  5.9,  5.8,  5.7,
        5.6,  5.5,  5.4,  5.3,  5.2,  5.1,  5. ,  4.9,  4.8,  4.7,  4.6,
        4.5,  4.4,  4.3,  4.2,  4.1,  4. ,  3.9,  3.8,  3.7,  3.6,  3.5,
        3.4,  3.3,  3.2,  3.1,  3. ,  2.9,  2.8,  2.7,  2.6,  2.5,  2.4,
        2.3,  2.2,  2.1,  2. ,  1.9,  1.8,  1.7,  1.6,  1.5,  1.4,  1.3,
        1.2,  1.1,  1. ,  0.9,  0.8,  0.7,  0.6,  0.5,  0.4,  0.3,  0.2,
        0.1])

In [ ]:
def p1(T, dt, data, v_max, L):
	p1 = np.zeroslike(data.getx()[:,0]) # anpassen
	t_arr = np.arange(T, 0, -dt)
	for i, j in enumerate(t_arr):
		if i == 0:
			p = 0
		else:
			p = p + dt * 2*(data.getnumx()-data.getxdata()) - dt * p * (v_max/d[-i,0])
		p1[-i] = p
	return p1

def p2_N_1(T,dt, data, pi_1, d, n): # n tes auto
	pi = np.zeroslike(data.getx()[:,1:-1]) # anpassen
	t_arr = np.arange(T, 0, -dt)
	for i, j in enumerate(t_arr):
		if i == 0:
			p = 0
		else:
			p = p + dt * 2*(data.getnumx()-data.getxdata()) - dt * p * (v_max/d[-i,0]) + dt * pi_1[-i] * v_max/d[-i, n]
		pi[-i] = p
	return pi

def pN(T,dt, data, pi_1, d, n): # n tes auto
	pi = np.zeroslike(data.getx()[:,1:-1]) # anpassen
	t_arr = np.arange(T, 0, -dt)
	for i, j in enumerate(t_arr):
		if i == 0:
			p = 0
		else:
			p = p + dt * 2*(data.getnumx()-data.getxdata()) + dt * pi_1[-i] * v_max/d[-i, n]
		pi[-i] = p
	return pi




In [ ]:
N = 5
def integral(arg, dt):
    res = [arg[i] * dt for i in range(len(arg))]
    return np.sum(res)

def gradient(P_ges, d, v_max, L):
    sk0 = integral(P_ges[-1] + np.sum([P_ges[i] * np.ln(d[i]/L) for i in range(P_ges.shape[0]-1)]))
    sk1 = integral(np.sum([P_ges[i] * v_max/L for i in range(P_ges.shape[0]-1)]))
    return (sk0, sk1)

sk = gradient
sigma = None # Amijo Bed.
for k in range(N):
    direction = sk
    # (update stepsize) bei uns fix
    v_max = v_max + sigma[0] * sk[0]
    L = L + sigma[1] * sk[1] 